In [1]:
#import segmentation_models_pytorch as smp
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import albumentations as albu
#import torch
import numpy as np
#from torch.utils.data import DataLoader
#from torch.utils.data import Dataset as BaseDataset
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
#将图片路径写成列表，为后面cv2读入方便
images_dir = 'liefeng'
ids=sorted(os.listdir(images_dir))
images_fps=[os.path.join(images_dir, image_id) for image_id in ids]

In [3]:
#批量处理，文件夹内20张照片读成np.array存入3维array‘image’，第一维代表第几张照片
j=0
image = np.zeros(shape=(len(images_fps),320,320))
for i in images_fps:
    image1 = cv2.imread(i,1)
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    #像素值变成0-1
    image2=image1[:,:,1]
    image2=image2-image2.min()
    image2=image2/image2.max()
    #照片周围变成空白像素，否则mat算法会报错
    image2[0,:]=0
    image2[image2.shape[0]-1,:]=0
    image2[:,0]=0
    image2[:,image2.shape[1]-1]=0
    image2=np.reshape(image2,(320,320))
    image[j,:,:]=image2
    j=j+1

In [5]:
#用于检查是否读入，cv2显式照片
cv2.namedWindow("Image",0)
cv2.imshow('Image',image[1,:,:])
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
#从复合图分离
'''
之前的复合图像
skeleton_img=skeleton_img*200#matplot为0-255绘图，0黑，255白。
skeleton_img_liefeng=skeleton_img+image*55
for n in range (0,19):
    #matplotlib.image.imsave("liefeng/%04.d.png"%(n+1),skeleton_img[n,:,:])
    cv2.imwrite("liefeng/%04.d.png"%(n+1),skeleton_img_liefeng[n,:,:])
'''
image_liefeng=np.zeros_like(image)
image_gujia=np.zeros_like(image)
image_gujia[image > 0.99]=1
image_liefeng[image >0.1]=1

In [7]:
Image_gujia=image_gujia*255#放大到0-255用于保存照片
Image_liefeng=image_liefeng*255#放大的到0-255用于保存照片

In [6]:
#用于检查是否读入，cv2显式照片,0=黑，1=白
cv2.namedWindow("Image",0)
cv2.imshow('Image',image_liefeng[1,:,:])
cv2.waitKey()
cv2.destroyAllWindows()

In [56]:
#opencv保存成照片(0-255，因此需要先放大255倍)，放在liefeng文件夹
for n in range (0,19):
    #matplotlib.image.imsave("liefeng/gujia%04.d.png"%(n+1),Image_gujia[n,:,:])
    cv2.imwrite("gujia/%04.d.png"%(n+1),Image_gujia[n,:,:])
    cv2.imwrite("liefeng2/%04.d.png"%(n+1),Image_liefeng[n,:,:])

In [9]:
def get_gujia_point(image):
    "接受骨架灰度照片(0-1)，返回2维ndarray，记录所有骨架像素点位置"
    gujia_P=np.nonzero(image)
    gujia_p=np.array(gujia_P)
    return gujia_p
gujia_p4=get_gujia_point(image_gujia[1,:,:])

In [10]:
#返回一张照片的所有骨架像素点位置
def get_gujia_point(image):
    "接受骨架灰度照片(0-1)，返回2维ndarray，记录所有骨架像素点位置"
    gujia_P=np.nonzero(image)
    gujia_p=np.array(gujia_P)
    return gujia_p
gujia_p4=get_gujia_point

#返回所有框选宽度
def get_w(image_liefeng,image_gujia,point,w,h):
    "输入image裂缝照片，point骨架点(2dimension，第一维代表纵坐标，第二位横坐标），w框选半宽，h框选半高，返回所有框选出照片的所有宽度，存放在ndarray'list中'"
    n=point.shape[1]
    list=[]
    for i in range(0,n-1):
        #防止-h，-w后下标越界；上界可以越没关系
        if point[0,i]-h>=0 and point[1,i]-w>=0:
            img_for_w=image_liefeng[point[0,i]-h:point[0,i]+h,point[1,i]-w:point[1,i]+w]
            gj_for_w=image_gujia[point[0,i]-h:point[0,i]+h,point[1,i]-w:point[1,i]+w]
        elif point[0,i]-h>=0 and point[1,i]-w<0:
            img_for_w=image_liefeng[point[0,i]-h:point[0,i]+h,0:point[1,i]+w]
            gj_for_w=image_gujia[point[0,i]-h:point[0,i]+h,0:point[1,i]+w]
        elif point[0,i]-h<0 and point[1,i]-w>=0:
            img_for_w=image_liefeng[0:point[0,i]+h,point[1,i]-w:point[1,i]+w]
            gj_for_w=image_gujia[0:point[0,i]+h,point[1,i]-w:point[1,i]+w]
        else:
            img_for_w=image_liefeng[0:point[0,i]+h,0:point[1,i]+w]
            gj_for_w=image_gujia[0:point[0,i]+h,0:point[1,i]+w]
        width=calculate_width(img_for_w,gj_for_w)
        list.append(width)
    return list
#给出骨架和裂缝图片，统计点数，计算框选宽度
def calculate_width(image_for_w,gujia_for_w):
    "img_for_w框选的裂缝图，gj_for_w框选的骨架图，最后返回计算的平均宽度"
    area=np.count_nonzero(image_for_w)
    length=np.count_nonzero(gujia_for_w)
    width=area/(length)
    return width

def get_width(image_liefeng,image_gujia,w,h):
    gujia_p=get_gujia_point(image_gujia)
    width=get_w(image_liefeng,image_gujia,gujia_p,w,h)
    return width
#width=get_width(image_liefeng[1,:,:],image_gujia[1,:,:],10,10)
#width[400:450]
#img_for_w=get_img_for_w(image_liefeng[3,:,:],gujia_p4,w,h) 

In [11]:
w=10
h=10
width_all=[]
for i in range(19):
    width=get_width(image_liefeng[i,:,:],image_gujia[i,:,:],30,30)
    width_sort=sorted(width,reverse=True)
    width_sort=width_sort[0:20]
    width_all.append(width_sort)

[24.974358974358974,
 24.833333333333332,
 24.833333333333332,
 24.79746835443038,
 24.705128205128204,
 24.705128205128204,
 24.442307692307693,
 24.345454545454544,
 24.240506329113924,
 24.20689655172414,
 24.088607594936708,
 24.088607594936708,
 24.0327868852459,
 24.024390243902438,
 24.024390243902438,
 23.924050632911392,
 23.84375,
 23.759493670886076,
 23.759493670886076,
 23.73076923076923]

In [43]:
width_sort=sorted(width,reverse=True)
width20_sort=sorted(width20,reverse=True)
width30_sort=sorted(width30,reverse=True)
width40_sort=sorted(width40,reverse=True)
print(width_sort[0:20],'\n\n',width20_sort[0:20],'\n\n',width30_sort[0:20],'\n\n',width40_sort[0:20])

[23.1875, 22.88235294117647, 22.266666666666666, 22.11111111111111, 21.357142857142858, 21.05263157894737, 20.46153846153846, 20.272727272727273, 20.25, 20.23076923076923, 20.214285714285715, 20.2, 20.1875, 20.176470588235293, 20.166666666666668, 20.157894736842106, 20.0, 20.0, 20.0, 20.0] 

 [39.54545454545455, 39.523809523809526, 34.375, 34.31707317073171, 33.743589743589745, 33.38095238095238, 33.333333333333336, 33.333333333333336, 33.208333333333336, 33.125, 33.041666666666664, 32.86842105263158, 32.6530612244898, 32.6530612244898, 32.6530612244898, 32.6530612244898, 32.6530612244898, 32.63265306122449, 32.61224489795919, 32.59183673469388] 

 [43.465753424657535, 43.361111111111114, 43.16216216216216, 43.082191780821915, 43.02777777777778, 42.0958904109589, 41.753246753246756, 41.16216216216216, 40.24, 39.78048780487805, 39.46341463414634, 39.328947368421055, 38.44155844155844, 38.03896103896104, 37.724137931034484, 37.166666666666664, 36.31645569620253, 36.285714285714285, 35.56

In [34]:
#

a
b


In [18]:
cv2.namedWindow("Image",0)
cv2.imshow('Image',img_for_w[1])
cv2.waitKey()
cv2.destroyAllWindows()
#gj_for_w=get_img_for_w(image_gujia,gujia_p,w,h)

NameError: name 'img_for_w' is not defined

In [159]:
cv2.namedWindow("Image",0)
cv2.imshow('Image',img_for_w[0])
cv2.waitKey()
cv2.destroyAllWindows()

In [162]:
a=np.array([1,1])
b=np.array([[1,1],[1,1]])
list=[a,b]
list[1]
a=np.append(a,1)
a.shape

(3,)

In [140]:
img_for_w=img_for_w*255
cv2.imwrite("kuandu/%04.d.png"%(1),img_for_w)

True

In [141]:
print(area,'   ',length,'    ',width)

117     28      4.178571428571429


In [ ]:
(16,19)
(19,23)